In [ ]:
pip install ta mplfinance

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd 
import seaborn as sns
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator, EMAIndicator
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# Load dataset
btc_data = pd.read_csv("/kaggle/input/crypto-currencies-daily-prices/BTC.csv")
btc_data['date'] = pd.to_datetime(btc_data['date'])
btc_data.set_index('date', inplace=True)
print(btc_data.head())
# Add technical indicators
btc_data['SMA_7'] = SMAIndicator(btc_data['close'], window=7).sma_indicator()
btc_data['EMA_12'] = EMAIndicator(btc_data['close'], window=12).ema_indicator()
btc_data['EMA_26'] = EMAIndicator(btc_data['close'], window=26).ema_indicator()
btc_data['RSI'] = RSIIndicator(btc_data['close']).rsi()
bbands = BollingerBands(btc_data['close'])
btc_data['Bollinger_High'] = bbands.bollinger_hband()
btc_data['Bollinger_Low'] = bbands.bollinger_lband()

# Drop rows with NaN values due to indicator calculations


FileNotFoundError: [Errno 2] No such file or directory: 'BTC.csv'

In [ ]:
# Recalculate MACD and Signal Line
btc_data['MACD'] = btc_data['EMA_12'] - btc_data['EMA_26']
btc_data['Signal_Line'] = btc_data['MACD'].rolling(window=9).mean()
btc_data['MACD_Histogram'] = btc_data['MACD'] - btc_data['Signal_Line']
btc_data = btc_data.dropna()

In [ ]:
print(btc_data.head())
print(btc_data.tail())

In [ ]:
btc_data.describe()

In [ ]:
btc_data.info()

In [ ]:
btc_data.isnull().sum()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(btc_data['close'], label='Close Price', color='blue')
plt.title('Bitcoin Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.legend()

In [ ]:
btc_data['Trend'] = np.where(btc_data['close'] > btc_data['EMA_12'], 'Bullish', 'Bearish')
plt.figure(figsize=(12, 6))
plt.plot(btc_data['close'], label='Close Price', color='blue')
plt.plot(btc_data['EMA_12'], label='EMA 12', color='green', linestyle='--')
plt.plot(btc_data['EMA_26'], label='EMA 26', color='red', linestyle='--')
plt.fill_between(btc_data.index, btc_data['close'], color='green', where=btc_data['Trend'] == 'Bullish', alpha=0.1, label='Bullish Zone')
plt.fill_between(btc_data.index, btc_data['close'], color='red', where=btc_data['Trend'] == 'Bearish', alpha=0.1, label='Bearish Zone')
plt.title('Overlay Trend Zones')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()

In [ ]:
last_year_data = btc_data.loc[btc_data.index >= (btc_data.index.max() - pd.DateOffset(years=1))]
mpf_data = last_year_data.reset_index()
mpf_data.set_index('date', inplace=True)
mpf.plot(mpf_data, type='candle', mav=(7, 12, 26), volume=False, title='Candlestick Chart with SMA & EMA', style='yahoo')

In [ ]:
# Visual 4: Histogram for MACD

plt.figure(figsize=(12, 6))
plt.plot(last_year_data['MACD'], label='MACD Line', color='blue')
plt.plot(last_year_data['Signal_Line'], label='Signal Line', color='red', linestyle='--')
plt.bar(last_year_data.index, last_year_data['MACD_Histogram'], label='MACD Histogram', color='green', alpha=0.6)
plt.title('MACD and Signal Line with Histogram')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(btc_data['close'], bins=50, kde=True, color='blue')
plt.title('Distribution of Bitcoin Closing Prices')
plt.xlabel('Price (USD)')
plt.ylabel('Frequency')

In [ ]:
btc_data['month'] = btc_data.index.month
monthly_avg = btc_data.groupby('month')['close'].mean()
plt.figure(figsize=(10, 6))
monthly_avg.plot(kind='bar', color='blue')
plt.title('Average Bitcoin Prices by Month')
plt.xlabel('Month')
plt.ylabel('Average Price (USD)')

In [ ]:
numeric_data = btc_data.select_dtypes(include=['number'])
corr = numeric_data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Feature Correlation Heatmap')

In [ ]:
features = ['close', 'SMA_7', 'EMA_12', 'EMA_26', 'RSI', 'MACD', 'Signal_Line']
target = 'close'
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(btc_data[features])
scaled_target = scaler.fit_transform(btc_data[[target]])

X, y = [], []
window_size = 45  # Use the past 30 days for predictions
for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i-window_size:i])
    y.append(scaled_target[i])
X, y = np.array(X), np.array(y)

# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]



In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1)

# Make predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)



In [ ]:
# Calculate accuracy percentage
accuracy = 100 - np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(accuracy)

In [ ]:
plt.plot(range(len(y_test)), y_test, label='Actual', color='blue')
plt.plot(range(len(y_pred)), y_pred, label='Predicted', color='orange')
plt.title('Actual vs Predicted Prices')
plt.xlabel('Index')
plt.ylabel('Close Price (USD)')

In [ ]:

###########################################
# 1) Download BTC Data from Yahoo Finance
###########################################
# Define the start and end date for data extraction
start_date = "2017-01-01"
end_date = dt.datetime.today().strftime('%Y-%m-%d')

# Download BTC-USD data
df = yf.download("BTC-USD", start=start_date, end=end_date, interval='1d')
df.dropna(inplace=True)

# Clean and rename columns
df.reset_index(inplace=True)
df.rename(columns={
    'Date': 'timestamp',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

# Re-order and sort
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
df.sort_values('timestamp', inplace=True)
df.reset_index(drop=True, inplace=True)

###########################################
# 2) Add Technical Indicators
###########################################
# Compute some sample technical indicators (SMA, RSI, MACD)
import ta
df['sma_14'] = ta.trend.SMAIndicator(close=df['close'], window=14).sma_indicator()
df['rsi_14'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
macd = ta.trend.MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_diff'] = macd.macd_diff()

# Drop rows with NaN values resulting from indicator calculations
df.dropna(inplace=True)

###########################################
# 3) Prepare the Data for LSTM
###########################################
# Scale the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'volume', 'sma_14', 'rsi_14', 'macd', 'macd_signal', 'macd_diff']])

# Create sequences for LSTM
sequence_length = 60  # Use the past 60 days to predict the next day
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])  # Past 60 days
    y.append(scaled_data[i, 3])  # 'close' price as the target

X = np.array(X)
y = np.array(y)

# Train-test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

###########################################
# 4) Build the LSTM Model
###########################################
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))  # Output layer

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))

###########################################
# 5) Predict the Next Day's Price
###########################################
# Use the last `sequence_length` data points to predict the next day
last_sequence = scaled_data[-sequence_length:]
last_sequence = np.expand_dims(last_sequence, axis=0)

# Predict the next day's scaled price
next_day_scaled = model.predict(last_sequence)
next_day_price = scaler.inverse_transform([[0, 0, 0, next_day_scaled[0][0], 0, 0, 0, 0, 0, 0]])[0][3]

# Determine the next day's date
last_date = df['timestamp'].max()
next_date = last_date + pd.Timedelta(days=1)

print(f"\nPredicted Closing Price for {next_date.strftime('%Y-%m-%d')}: ${next_day_price:.2f}")

###########################################
# 6) Visualization
###########################################
plt.figure(figsize=(12, 6))
plt.plot(df['timestamp'], df['close'], label='Historical Close Prices', color='blue')
plt.axvline(x=last_date, color='black', linestyle='--', label='Last Known Date')
plt.scatter(next_date, next_day_price, color='red', label='Next Day Prediction')
plt.xlabel('Date')
plt.ylabel('BTC Price (USD)')
plt.title('Next Day Price Prediction with LSTM')
plt.legend()
plt.show()

In [ ]:
df.tail()